In [4]:
import copy
import os
import sys

import numpy as np
import open3d as o3d

In [5]:
source = o3d.io.read_point_cloud("src.ply")
target = o3d.io.read_point_cloud("dst.ply")

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [7]:
threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)

In [13]:
# point-to-point icp

print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)
print(f"evaluation_transformation=\n{evaluation.transformation}")

Initial alignment
RegistrationResult with fitness=1.747228e-01, inlier_rmse=1.177106e-02, and correspondence_set size of 34741
Access transformation to get result.
evaluation_transformation=
[[ 0.862  0.011 -0.507  0.5  ]
 [-0.139  0.967 -0.215  0.7  ]
 [ 0.487  0.255  0.835 -1.4  ]
 [ 0.     0.     0.     1.   ]]


In [16]:
print("Apply point-to-point ICP")

reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPoint()
)
print(reg_p2p)
print(f"reg_p2p.transformation=\n{reg_p2p.transformation}")

Apply point-to-point ICP
RegistrationResult with fitness=3.724495e-01, inlier_rmse=7.760179e-03, and correspondence_set size of 74056
Access transformation to get result.
reg_p2p.transformation=
[[ 0.83924644  0.01006041 -0.54390867  0.64639961]
 [-0.15102344  0.96521988 -0.21491604  0.75166079]
 [ 0.52191123  0.2616952   0.81146378 -1.50303533]
 [ 0.          0.          0.          1.        ]]


In [20]:
print("Apply point-to-plane ICP")

reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init, o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print(reg_p2l)
print(f"reg_p2l.transformation=\n{reg_p2l.transformation}")

np.savetxt("reg_p2l.txt", reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=6.209722e-01, inlier_rmse=6.581453e-03, and correspondence_set size of 123471
Access transformation to get result.
reg_p2l.transformation=
[[ 0.84023324  0.00618369 -0.54244126  0.64720943]
 [-0.14752342  0.96523919 -0.21724508  0.81018928]
 [ 0.52132423  0.26174429  0.81182576 -1.48366001]
 [ 0.          0.          0.          1.        ]]


In [34]:
# validation
print(p:=source.points[0])
print(q:=target.points[0])
# q_h = 
print(p.shape)
print(q.shape)

p_h = np.ndarray(p.shape[0]+1)
p_h[:3] = p
p_h[3] = 1.0
q_h = np.ndarray(q.shape[0]+1)
q_h[:3] = q
q_h[3] = 1.0
print(q_h)
Tq_h = reg_p2l.transformation.dot(q_h)
print(Tq_h)

[1.01953125 0.88671875 2.27726722]
[1.32421875 1.16796875 1.39391088]
(3,)
(3,)
[1.32421875 1.16796875 1.39391088 1.        ]
[1.01096963 1.43938493 0.64400922 1.        ]


In [37]:
# generate target point cloud by transforming source cloud
target_from_source = copy.deepcopy(source)
target_from_source.transform(reg_p2l.transformation)
if not os.path.exists("dst.ply"):
    o3d.io.write_point_cloud("dst.ply", target_from_source)

True